
#  Frisch-Waugh-Lovell and partialling out.
This notebook provides an empirical illustration of the Frisch-Waugh-Lovell and partialling out as studied in the course notes.



## Review of the Frisch-Waugh-Lovell and partialling out.

 Let us first summarize the main theoretical result.

We consider the following equation:
$$
\begin{align}
Y &= X^\top\beta + U =  D^\top\beta_1 + W^\top\beta_2 + U, \quad \operatorname{E}(XU) = 0
\end{align}
$$
Where $Y$ is the outcome/dependent variable, $X:=(D^\top, W^\top)^\top)$ is a $(k_1+k_2)\times 1$ vector of regressors, $\beta:=(\beta_1^\top, \beta_2^\top)^\top$ are parameters. We partition the vector
of regressors $X$ into two groups: the $k_1$-dimensional subvector $D$ represents “target” regressors of interest, and $k_2$-dimensional subvector $W$ represents other regressors, sometimes called the controls. For example, in wage gender gap analysis, where $Y$ is wage, $D$ is the gender indicator, and $W$ are various other variables explaining variation in wages. In program evaluation, $D$ is often a treatment or policy variable and $W$ are controls. $U$ represent unobserved determinants of $Y$, i.e, the regression error.

In population, define the partialling-out operator with respect to a vector $W$ that takes a
random variable $V$ such that $\operatorname{E}(V^2)<\infty$  and creates $\tilde{V}$ according to the rule:
$$
\begin{align*}
\tilde{V} = V - W^\top\gamma_{V, W}, \quad  \gamma_{V, W} =\argmin_{b\in \R^{k_2}} \operatorname{E}\left((V - W^\top b)^2\right).
\end{align*}
$$
When $V$ is a vector, we interpret the application of the operator as componentwise. The
vector $W$ needs to have finite second moment in order for this to be well-defined.

A remarkable fact, known as Frisch-Waugh-Lovell (FWL) theorem asserts
that $\beta_1$ is a regression coefficient of $Y$ on $D$ after partialing-out the linear effect of $W$
from $Y$ and $D$. In other words, it measures linearly the predictive effect (PE) of $D$ on
$Y$ , after taking out the linear predictive effect of $W$ on both of these variables:

\begin{align*}
\beta_1 &= \argmin_{b\in \R^{k_1}} \operatorname{E}\left((\tilde{Y} - \tilde{D}^\top b)^2\right) = 
\left(\operatorname{E}(\tilde{D}\tilde{D}^\top)\right)^{-1}\operatorname{E}(\tilde{D}\tilde{Y}).
\end{align*}

In the sample, partialling-out operation works similarly. Define it as an operator that
converts $V_i$ into $\check{V}_i$ via

\begin{align*}
\check{V}_i &= V_i - W_i^\top\hat{\gamma}_{V, W}, \quad \hat{\gamma}_{V, W} = \argmin_{b\in \R^{k_2}}\operatorname{E}n\left(V-W^\top b)^2\right).
\end{align*}

assuming that we have a sample of $n$ i.i.d. copies $\{(V_i, W_i^\top)\}_{i=1}^n$ of $(V, W^\top)$.

Similarly to the population case, we have the sample version of the FWL Theorem:

\begin{align*}
\hat{\beta}_1 = \argmin_{b\in \R^{k_1}} \operatorname{E}_n\left((\check{Y} - \check{D}^\top b)^2\right) = \left(\operatorname{E}_n( \check{D} \check{D}^\top)\right)^{-1}\operatorname{E}_n( \check{D} \check{Y}),
\end{align*}

where the notion $\operatorname{E}_n\left( f(W) \right)$ abbreviates the
empirical expectation of $f(W)$ as $W$ ranges over the sample $(W_i)_{i=1}^n$:
\begin{align*}
\operatorname{E}_n\left( f(W) \right)&= n^{-1}\sum_{i=1}^n f(W_i),
\end{align*}


## Application to the gender wage gap in France.

### Data Description

We consider an empirical application to gender wage gap using data from the French Enployment Survey in 2012. The subsample used in the analysis concerns employed individuals, men and women. We drop observations with missing values for at least one of the variables used in the analysis(outcome and regressors). 

Specifically the variable of interest $Y$ is the logarithm of the hourly wage rate constructed as the ratio of the annual earnings(SALRED) to the total number of hours worked(NBHEUR). The "target" regressor of interest $D$ is an indicator for female worker. The vector of controls $W$ consist of the following variables: indicators/dummies for the marital status(MATRI), for education(DDIPL), occupation(CSER), seniority(ANCENTR4), industry(NAFG4N), region(REG). We also include all the two-way interactions between the these variables. Finally, a quartic in potential experience constructed as
the age in 2012 minus the age at the end of schooling minus. More details are provided in the code in the next cells.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm 
from sklearn import (
    linear_model, metrics, neural_network, pipeline, preprocessing, model_selection
)

In [4]:
#url='https://drive.google.com/file/d/0B6GhBwm5vaB2ekdlZW5WZnppb28/view?usp=sharing'
#url='https://drive.google.com/uc?id=' + url.split('/')[-2]
#ee12_extract = pd.read_csv(url)
#ee12_extract.info()
ee12_extract = pd.read_csv('/Users/michalurdanivia/Library/CloudStorage/GoogleDrive-mw.urdanivia@gmail.com/Mon Drive/eemploi2012extract/eemploi2012_s0.csv', low_memory=False)
ee12_extract.info()
# Les données sont ici: 
# https://drive.google.com/drive/folders/1qluzqqon8_sF9B0RJ1eqRIqUld0xbBW6?usp=sharing
# Les télécharger est changer l'emplacement dans le code ci-dessus.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31045 entries, 0 to 31044
Columns: 801 entries, Unnamed: 0 to regzus941
dtypes: float64(401), int64(358), object(42)
memory usage: 189.7+ MB


In [ ]:
# Change the path to the file "eemploi2012_s0.csv" if needed.
eemploi = pd.read_csv('/Users/michalurdanivia/Documents/Bibliothèque/Recherche/Temp/Papiers_etc/eemploitemp/eemploi2012_s0.csv', low_memory=False) 
eemploi.shape # dimensions du fichiers: nombre de lignes/observations x nombre de colonnes/variables.
eemploi = eemploi[['ACTEU', 'SEXE', 'DDIPL', 'FORDAT', 'MATRI', 'TYPMEN5', 'ZUS', 'REG','AG', 
             'AGQ', 'AGEQ','AGE','AG5','NBENF3', 'NBENF6', 'NBENF18', 'SALRED', 'NBHEUR', 
             'NATPERC','NATMERC', 'PAIPERC','PAIMERC','TRIM', 'RGI', 'IDENT', 'NOI', 
             'DIP', 'DIP11','CONTRA','CSER', 'CSPM', 'CSPP', 'ANNEE', 'CHPUB', 'NAFG4N', 
             'ANCENTR4', 'exper', 'adfe', 'lsalhor']]

# Information on the variables selected from the extract(see the dictionary).
eemploi.info()

### Sample selection and additional variables.
We select a sample according to the criteria indicated above and define some additional variables, in 
particular log hourly wage, potential experience, and dummies/indicators for categorical variables. 

In [ ]:
# We consider the subsample of employed individuals.
df = eemploi[(eemploi.ACTEU == 1) & (eemploi.NAFG4N !='00') & (eemploi.CSER !=0)]

# We drop observations with missing values.
df = df.dropna()

# Log-Hourly wage.
df['lsal'] = np.log(df['SALRED']/df['NBHEUR'])

# Age at the end of schooling.
df['afe'] = df['FORDAT'] - (2012 - df['AG'])

# Approximate measure of experience in labor market.
df['exp'] = df['AG'] - df['afe']

# Dummies for categorical variables.
# These will not be used when building the design matrix of regressors from the patsy library(see below)
dummies = df[['DDIPL', 'MATRI', 'REG', 'ZUS', 'CSER', 'NAFG4N', 'ANCENTR4', 'SEXE']]
dummies = pd.get_dummies(data = dummies, columns=['DDIPL', 'MATRI', 'REG', 'ZUS', 'CSER', 'NAFG4N', 'ANCENTR4', 'SEXE'])
df = pd.concat([df, dummies], axis = 1)

# Some descriptive statistics for the variables used in the analysis.
# We first define a list with the hourly wage and potential experience and the dummies(hence 
# we use the dummies for computing frequencies for categorical variables).
listvar = ['lsal', 'exp'] + list(dummies)
df[listvar].describe()
#df.info()

### Wages, gender, education, occupation.
We now look into how (mean)wage is split across groups defined by gender, education, occupation.

In [ ]:
lsal_tab = df.groupby(["DDIPL", "SEXE", "CSER"])['lsal'].mean().unstack(level="SEXE")
lsal_tab

In [ ]:
#del(dummies, listvar)

## Regression analysis

We first estimate the linear regression model by OLS of $Y$ on $D$ with and without the vector of controls $W$. We then obtains the same estimate using the Frisch-Waugh-Lovell theorem for partialing-out the controls
via OLS. Finally we obtain the coefficient of $D$ using a variant of this procedure in
that partials-out the controls via LASSO instead of OLS. The Lasso estimator is a high dimensional regression method of particular interest for applications where the number of regressors is potentially bigger than the number of observations. Here we use this method in the (first step) regression of $D$ and $Y$ on $W$ in order to select the most predictive regressors. The lasso will be studied later in the course.

In [ ]:
# We use the "patsy" library for building the design matrix of regressors from an "R-type" formula.
import patsy
formula = """
lsal ~ SEXE_2
       + (C(DDIPL) + C(ANCENTR4) + C(NAFG4N) + C(REG) + C(CSER) + C(MATRI))**2 
       + exp + I(exp**2) + I(exp**3) + I(exp**4)      
"""
Y, X = patsy.dmatrices(formula, df, return_type = "dataframe")

In [ ]:
# Homoskedastic Standard Errors.
# linreg_ols0 = sm.OLS(endog = Y, exog = X, missing = 'drop').fit()
# print(linreg_ols0.summary()) 

# OLS with Robust "White" Standard Errors.
# Specification without controls.
linreg_ols0 = sm.OLS(endog = Y, exog = X[['Intercept','SEXE_2']], missing = 'drop').fit(cov_type='HC0')

# Default printing.
#print(linreg_ols0.summary())

# Specification with controls.
linreg_ols1 = sm.OLS(endog = Y, exog = X, missing = 'drop').fit(cov_type='HC0')

# Default printing.
#print(linreg_ols1.summary())

# We use Stargazer for printing the results(coefficients, and standard errors):
# only the coefficient for SEXE_2.
# See here for examples: https://github.com/mwburke/stargazer/blob/master/examples.ipynb
from stargazer.stargazer import Stargazer, LineLocation
stargazer = Stargazer([linreg_ols0 , linreg_ols1])
stargazer.covariate_order(['SEXE_2'])
stargazer

In [ ]:
# Application of Partialling Out/ Frisch-Waugh-Lovell Theorem(0)

W = X.drop(columns = ['SEXE_2'])
D = df['SEXE_2']
Y = df['lsal']

# Step 1: 
# we partial out the regressors/controls for the treatmement/policy variable and for the outcome.
Dreg = sm.OLS(endog = D, exog = W, missing = 'drop').fit(cov_type='HC0')
Dhat = Dreg.predict()
Dres = D - Dhat
Yreg = sm.OLS(endog = Y, exog = W, missing = 'drop').fit(cov_type='HC0')
Yhat = Yreg.predict()
Yres = Y - Yhat

# Step 2: 
# regression of the residuals associated to the regression of the outcome 
# on the residuals associated to the treatment/policy.
Y_partialReg = sm.OLS(endog = Yres, exog = Dres, missing = 'drop').fit(cov_type='HC0')
print(Y_partialReg.summary())

In [ ]:
# Application of Partialling Out/ Frisch-Waugh-Lovell Theorem(1)
# Difference with (0) is that we partial-out the controls via LASSO instead of OLS.

# OLS with sklearn: previously made with statsmodels.
#lr_model = linear_model.LinearRegression()
#lr_model.fit(X1, Y1)

# Step 1: Partial out the controls via Lasso with cross-validation.
Ylasso_model = linear_model.LassoCV(cv = 5)
Ylasso_model.fit(W, Y)
Yhat_lasso = Ylasso_model.predict(W)
Yres_lasso = Y - Yhat_lasso
Dlasso_model = linear_model.LassoCV(cv = 5)
Dlasso_model.fit(W, D)
Dhat_lasso = Dlasso_model.predict(W)
Dres_lasso = D - Dhat_lasso

# Step 2: OLS.
Y_partialReg_lasso = sm.OLS(endog = Yres_lasso, exog = Dres_lasso, missing = 'drop').fit(cov_type='HC0')
print(Y_partialReg_lasso.summary())

In [ ]:
import matplotlib.colors as mplc
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from pandas_datareader import DataReader

%matplotlib inline
# activate plot theme
import qeds
qeds.themes.mpl_style();

In [ ]:
def single_scatter_plot(df, DDIPL, sexe, ax, color):
    """
    This function creates a single year's and education level's
    log density to log wage plot
    """
    # Filter data to keep only the data of interest
    _df = df.query("(DDIPL == @DDIPL) & (SEXE == @sexe)")
    _df.plot(
        kind="scatter", x="exp", y="lsal", ax=ax, color=color
    )

    return ax

# Create initial plot
fig, ax = plt.subplots(1, 6, figsize=(16, 6), sharey=True)

for i in [1, 3, 4, 5, 6, 7]:
    single_scatter_plot(df, DDIPL, 1, ax[i], "b")
    single_scatter_plot(df, DDIPL, 2, ax[i], "r")
    ax[i].set_title(str(DDIPL))


In [ ]:
df2 = pd.read_csv("https://datascience.quantecon.org/assets/data/density_wage_data.csv")
df2.info()

In [ ]:
df2.info()

In [ ]:
df2.head()